# Identity matrix vs. Hessian matrix

18 April 2025

In this notebook, we compare two matrix types in `IntegrationProfiler`:
- matrix_type = :hessian,
- matrix_type = :identity (with 4 choices of gamma: 1e-1, 1e+0, 1e+2, 1e+4).

Our criteria for comparison are:
- Number of function evaluations
- Running time

### 1. Load packages

In [1]:
using Revise
using LikelihoodProfiler
using Optimization, ForwardDiff, OrdinaryDiffEq
using CSV, DataFrames, PrettyTables

### 2. Check for simple analytic functions

In a first experiment, we run different methods for simple analytic functions.

In [2]:
include(joinpath(@__DIR__, "../models/AnalyticFuncs/analytic_funcs.jl"))

Dict{Symbol, Dict{Symbol, Any}} with 8 entries:
  :f_5p       => Dict(:threshold=>4.0, :profile_range=>[(-20.0, 20.0), (-20.0, …
  :f_3p_1im   => Dict(:grad!=>#23, :threshold=>4.0, :profile_range=>[(-20.0, 20…
  :f_4p_2im   => Dict(:grad!=>#29, :threshold=>4.0, :profile_range=>[(-20.0, 20…
  :f_1p_ex    => Dict(:grad!=>#14, :threshold=>4.0, :profile_range=>[(-20.0, 20…
  :f_3p_1im2  => Dict(:grad!=>#26, :threshold=>4.0, :profile_range=>[(-20.0, 20…
  :f_2p_1im   => Dict(:grad!=>#17, :threshold=>4.0, :profile_range=>[(-20.0, 20…
  :f_2p       => Dict(:grad!=>#20, :threshold=>4.0, :profile_range=>[(-20.0, 20…
  :rosenbrock => Dict(:threshold=>4.0, :profile_range=>[(-10.0, 10.0), (-10.0, …

In [3]:
integrator = Tsit5()
integrator_opts = (dtmax=0.3,)

(dtmax = 0.3,)

In [5]:
for (name, infos) in funcs_dict
    optf = OptimizationFunction(infos[:func], AutoForwardDiff())
    optprob = OptimizationProblem(optf, infos[:optim])
    plprob = PLProblem(
        optprob, 
        infos[:optim], 
        infos[:profile_range]; 
        threshold = infos[:threshold]
    )

    method1 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :hessian)
    method2 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e-1)
    method3 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e+0)
    method4 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e+2)
    method5 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e+4)
    methods = [method1, method2, method3, method4, method5]
    
    time1 = @elapsed sol1 = profile(plprob, method1)
    time2 = @elapsed sol2 = profile(plprob, method2)
    time3 = @elapsed sol3 = profile(plprob, method3)
    time4 = @elapsed sol4 = profile(plprob, method4)
    time5 = @elapsed sol5 = profile(plprob, method5)
    sols = [sol1, sol2, sol3, sol4, sol5]
    times = [time1, time2, time3, time4, time5]

    df = []
    for (method, sol, time) in zip(methods, sols, times)
        push!(df, [
            (matrix_type=method.matrix_type, gamma=method.gamma), 
            map(i -> (s=sol[i].stats; s[1].nf+s[2].nf), 1:length(sol)),
            time
        ])
    end

    pretty_table(
        permutedims(reduce(hcat, df)),
        header=["method", "fevals", "runtime"],
        title=string(name)
    )
end

f_5p
┌────────────────────────────────────────────┬───────────┬───────────┐
│                                     method │    fevals │   runtime │
├────────────────────────────────────────────┼───────────┼───────────┤
│      (matrix_type = :hessian, gamma = 1.0) │ [90, 126] │ 0.0012738 │
│     (matrix_type = :identity, gamma = 0.1) │ [72, 126] │ 0.0005533 │
│     (matrix_type = :identity, gamma = 1.0) │ [72, 126] │  0.000511 │
│   (matrix_type = :identity, gamma = 100.0) │ [72, 126] │ 0.0004976 │
│ (matrix_type = :identity, gamma = 10000.0) │ [72, 126] │  0.000491 │
└────────────────────────────────────────────┴───────────┴───────────┘
f_3p_1im
┌────────────────────────────────────────────┬───────────────────────┬───────────┐
│                                     method │                fevals │   runtime │
├────────────────────────────────────────────┼───────────────────────┼───────────┤
│      (matrix_type = :hessian, gamma = 1.0) │       [126, 150, 816] │ 0.0060004 │
│     (matrix_t

- For two systems, `f_3p_1im` and `rosenbrock`, the number of function evaluations increases by a lot for larger gammas, and (seems to) lead to slower running time.
- For other systems, the number of function evaluations is stable.

### 3. Check for SIR and Taxol

In [2]:
integrator = FBDF(autodiff = AutoFiniteDiff())
integrator_opts = (;)

NamedTuple()

In [6]:
include(joinpath(@__DIR__, "../test/test_taxol_model.jl"))


Test Summary:                                                               | Pass  Total     Time
Taxol model. Fixed-step OptimizationProfiler with derivative-free optimizer |   17     17  2m16.1s
Test Summary:                                      | Pass  Total   Time
Taxol model. IntegrationProfiler with full hessian |   17     17  59.1s


Test.DefaultTestSet("Taxol model. IntegrationProfiler with full hessian", Any[], 17, false, false, true, 1.744972298195e9, 1.744972357271e9, false, "c:\\data\\projects\\insysbio\\LikelihoodProfiler.jl\\test\\test_taxol_model.jl")

In [7]:
include(joinpath(@__DIR__, "../models/Taxol/taxol_model.jl"))
optf = OptimizationFunction(taxol_obj, Optimization.AutoForwardDiff())
optprob = OptimizationProblem(optf, p0)
profile_range = [
  (2., 30.),
  (2.0, 30.),
  (0.01, 0.6),
  (0.05, 5.),
  (30., 250.)
]
plprob = PLProblem(optprob, p0, profile_range; threshold = sigmasq*chi2_quantile(0.95, 5))

Profile Likelihood Problem. Profile type: parameter
Parameters' optimal values: 
5-element Vector{Float64}:
   8.317
   8.0959
   0.0582
   1.3307
 119.1363

In [8]:
method1 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :hessian)
method2 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e-1)
method3 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e+0)
method4 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e+2)
method5 = IntegrationProfiler(integrator = integrator, integrator_opts = integrator_opts, matrix_type = :identity, gamma = 1e+4)
methods = [method1, method2, method3, method4, method5]

sols = []
times = []
for method in methods
    try
        time = @elapsed sol = profile(plprob, method)
        push!(sols, sol)
        push!(times, time)
    catch e
        println(method, "\n  ", e)
    end
end

df = []
for (method, sol, time) in zip(methods, sols, times)
    push!(df, [
        (matrix_type=method.matrix_type, gamma=method.gamma), 
        map(i -> (s=sol[i].stats; s[1].nf+s[2].nf), 1:length(sol)),
        time
    ])
end

pretty_table(
    permutedims(reduce(hcat, df)),
    header=["method", "fevals", "runtime"],
    title=string("Taxol")
)

IntegrationProfiler{Nothing, @NamedTuple{}, FBDF{5, 0, AutoFiniteDiff{Val{:forward}, Val{:forward}, Val{:hcentral}, Nothing, Nothing, Bool}, Nothing, NLNewton{Rational{Int64}, Rational{Int64}, Rational{Int64}, Rational{Int64}}, typeof(OrdinaryDiffEqCore.DEFAULT_PRECS), Val{:forward}(), true, nothing, Nothing, Nothing, typeof(OrdinaryDiffEqCore.trivial_limiter!)}, @NamedTuple{}}(false, nothing, NamedTuple(), FBDF{5, 0, AutoFiniteDiff{Val{:forward}, Val{:forward}, Val{:hcentral}, Nothing, Nothing, Bool}, Nothing, NLNewton{Rational{Int64}, Rational{Int64}, Rational{Int64}, Rational{Int64}}, typeof(OrdinaryDiffEqCore.DEFAULT_PRECS), Val{:forward}(), true, nothing, Nothing, Nothing, typeof(OrdinaryDiffEqCore.trivial_limiter!)}(Val{5}(), nothing, NLNewton{Rational{Int64}, Rational{Int64}, Rational{Int64}, Rational{Int64}}(1//100, 10, 1//5, 1//5, false, true, 0//1), OrdinaryDiffEqCore.DEFAULT_PRECS, nothing, nothing, :linear, :Standard, OrdinaryDiffEqCore.trivial_limiter!, AutoFiniteDiff()), 

ArgumentError: ArgumentError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer